In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import swiftest
from astroquery.jplhorizons import Horizons

In [2]:
swiftersim = swiftest.Simulation(param_file="param.swifter.in", codename="Swifter")
swiftersim.bin2xr()
swifterdat = swiftersim.ds

Reading Swifter file param.swifter.in
Reading in time 1.000e+03
Creating Dataset
Successfully converted 1001 output frames.
Swifter simulation data stored as xarray DataSet .ds


In [3]:
swiftestsim = swiftest.Simulation(param_file="param.swiftest.in")
swiftestsim.bin2xr()
swiftestdat = swiftestsim.ds

Reading Swiftest file param.swiftest.in
Reading in time 1.000e+03
Creating Dataset
Successfully converted 1001 output frames.
Swiftest simulation data stored as xarray DataSet .ds


In [4]:
swifterdat['varpi'] = swifterdat['omega'] + swifterdat['capom']
swiftestdat['varpi'] = swiftestdat['omega'] + swiftestdat['capom']

In [5]:
obj = Horizons(id='1', id_type='majorbody',location='@sun',
               epochs={'start':'2021-01-28', 'stop':'3021-02-05',
                       'step':'1y'})
el = obj.elements()
t = (el['datetime_jd']-el['datetime_jd'][0]) / 365.25
varpi_obs = el['w'] + el['Omega']

In [6]:
varpiswiftest = swiftestdat['varpi'].sel(id=2) * 180.0 / np.pi
varpiswifter = swifterdat['varpi'].sel(id=2) * 180.0 / np.pi
tsim = swiftestdat['time']

In [7]:
dvarpi_swiftest = np.diff(varpiswiftest)  * 3600 * 100 
dvarpi_swifter = np.diff(varpiswifter)  * 3600 * 100 
dvarpi_obs = np.diff(varpi_obs) / np.diff(t) * 3600 * 100 

Mean precession rate for Mercury long. peri. (arcsec/100 y)
JPL Horizons      : 571.3210506300043
Swifter GR        : 571.1981012667947
Swiftest GR       : 1.5844780122245083
Obs - Swifter     : 0.12294936320971743
Obs - Swiftest    : 569.7365726177798
Swiftest - Swifter: -569.61362325457


In [8]:
fig, ax = plt.subplots()

ax.plot(t, varpi_obs, label="JPL Horizons")
ax.plot(tsim, varpiswifter, label="Swifter GR")
ax.plot(tsim, varpiswiftest, label="Swiftest GR")
ax.set_xlabel('Time (y)')
ax.set_ylabel('Mercury $\\varpi$ (deg)')
ax.legend()
print('Mean precession rate for Mercury long. peri. (arcsec/100 y)')
print(f'JPL Horizons      : {np.mean(dvarpi_obs)}')
print(f'Swifter GR        : {np.mean(dvarpi_swifter)}')
print(f'Swiftest GR       : {np.mean(dvarpi_swiftest)}')
print(f'Obs - Swifter     : {np.mean(dvarpi_obs - dvarpi_swifter)}')
print(f'Obs - Swiftest    : {np.mean(dvarpi_obs - dvarpi_swiftest)}')
print(f'Swiftest - Swifter: {np.mean(dvarpi_swiftest - dvarpi_swifter)}')